In [2]:
#load data
import pandas as pd

original_data_canton_FR = pd.read_excel(r'../dataset/raw/Dep_15_Resultats_T1_complet.xlsx', sheet_name='Cantons', header=2)
original_data_Bvot_FR        = pd.read_csv('../dataset/inputs/XDataFR_Bvot.csv', sep=';')
original_data_Bvot_targets_FR = pd.read_csv('../dataset/labels/yDataFR_Bvot.csv', sep=';')

data_canton_FR = original_data_canton_FR.copy()
data_Bvot_FR   = original_data_Bvot_FR.copy()
data_Bvot_targets_FR = original_data_Bvot_targets_FR.copy()

In [3]:
def saveData(data, loc):
    # save cher_data as excel
    writer = pd.ExcelWriter(loc)
    
    # write dataframe to excel
    data.to_excel(writer)

    # save the excel
    writer.save()

In [4]:
def getNbBinomes(data):
    return len([header for header in data.columns if "Binôme" in header])

def explodeLines(data):
    initdf = data[['Code du département', 'Libellé du département', 'Code du canton', 
            'Libellé du canton', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants',
            '% Vot/Ins', 'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins',
            '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot']]

    headers = ['N°Panneau', 'Nuance', 'Binôme', 'Sièges', 'Voix', '% Voix/Ins', '% Voix/Exp']
    
    df = pd.DataFrame()
    for i in range(getNbBinomes(data)):
        partidf = data[[h+'.'+str(i) if i!=0 else h for h in headers]]
        partidf = pd.concat([initdf, partidf], axis=1)
        partidf.columns = pd.Index(initdf.columns.values.tolist() + headers)
        df = pd.concat([df, partidf])
    
    # Remove useless rows   
    df = df.dropna(how='all', subset=headers)
    
    return df

In [5]:
#explode line 
data_canton_FR = explodeLines(data_canton_FR)

In [33]:
def getNuanceOfElected(data, col_siege='Sièges', col_nuance='Nuance'):
    elected = data[data[col_siege]=='Elus']
    return list(elected[col_nuance]) if len(elected)!=0 else None

def filterBestNuances(data, col_nuance='Nuance', criteria=12.50):
    bestCandidat = data[data['% Voix/Ins']>= criteria]
    if bestCandidat.empty or len(bestCandidat)==1:
        bestCandidat = data.sort_values(by='Voix', ascending=False).iloc[0:2,:]
    if all(data['Code du département']==str(14)) and all(data[col_nuance]==15):
        print(data[col_nuance])

    return list(bestCandidat[col_nuance])

def getDuels(data, dep, col_dep='Code du département', col_canton='Code du canton', col_siege='Sièges', col_nuance='Nuance',count=0):
    '''
        ATTENTION : data doit etre EXPLODE !
    '''
    data = data[data[col_dep]==dep]
    duels = dict()
    for canton in data[col_canton].unique():
        data_canton = data[data[col_canton]==canton]
        # allow to know if there is a majority in the canton
        elected = getNuanceOfElected(data_canton, col_siege=col_siege, col_nuance=col_nuance) 
        
        if elected is not None:
            count+=1
            duels[str(canton)]= elected
        else:
            duels[str(canton)] = filterBestNuances(data_canton)
            if len(duels[str(canton)])<2:
                print(f'dep/canton {(dep,canton)} duel {duels[str(canton)]}')
    return duels, count

def optimizeDuelDict(duels):
    optdic = dict()
    for dep, duelDepDict in duels.items():
        for canton, duelList in duelDepDict.items():
            key = ':'.join(duelList)
            if key in optdic.keys():
                optdic[key].append((dep, canton))
            else:
                optdic[key]= [(dep, canton)]
    return optdic

In [34]:
duels = dict()
count=0
for dep in data_canton_FR['Code du département'].unique():
    duels[str(dep)], count= getDuels(data_canton_FR, dep, count=count)
    if duels[str(dep)]==[]:
        print('empty list for dep : ', dep)

count


149

In [38]:
optDuels = optimizeDuelDict(duels)
win = [duel.split(':') for duel in list(optDuels.keys()) if len(duel.split(':'))<2]

l = []
for winner in win :
    l += optDuels[':'.join(winner)]

print(len(optDuels))

170


In [36]:
optDuels[':'.join(['BC-FN'])]
duels['14']


{'1': ['BC-FN', 'BC-DVD'],
 '2': ['BC-UD', 'BC-FN'],
 '3': ['BC-DVD', 'BC-FN'],
 '4': ['BC-UD', 'BC-FN'],
 '5': ['BC-SOC', 'BC-UD'],
 '6': ['BC-SOC', 'BC-UD'],
 '7': ['BC-UG', 'BC-UD'],
 '8': ['BC-SOC', 'BC-UD'],
 '9': ['BC-UG', 'BC-UD'],
 '10': ['BC-DVG', 'BC-DVD', 'BC-FN'],
 '11': ['BC-DVD', 'BC-SOC'],
 '12': ['BC-DVG', 'BC-UD'],
 '13': ['BC-SOC', 'BC-UDI', 'BC-FN'],
 '14': ['BC-SOC', 'BC-MDM'],
 '15': ['BC-DVD', 'BC-DVD'],
 '16': ['BC-SOC', 'BC-UD'],
 '17': ['BC-DVD', 'BC-FN'],
 '18': ['BC-UD', 'BC-FN'],
 '19': ['BC-DVD', 'BC-FN'],
 '20': ['BC-SOC', 'BC-UMP'],
 '21': ['BC-DVD'],
 '22': ['BC-SOC', 'BC-DVD', 'BC-FN'],
 '23': ['BC-UD', 'BC-FN'],
 '24': ['BC-UG', 'BC-UD', 'BC-FN'],
 '25': ['BC-DVG', 'BC-UMP']}

In [ ]:

# retourne un dataset 
def getTrainSets(X, y, duel, col_canton='CODCAN', col_dep='Code du département'):
    mask_dep    = pd.Series([False]*len(X))
    mask_canton = pd.Series([False]*len(X))
    for dep in X[col_dep].unique():
        for canton in X[X[col_dep]==dep][col_canton].unique(): #pour chaque canton du departement
            #on considere que les cantons qui ont comme partis les partis du duel
            if all(nuance in X.loc[(X[col_dep]==dep) & (X[col_canton]==canton)] for nuance in duel):
                mask_dep    |= X[col_dep]==dep
                mask_canton |= X[col_canton]==canton

    # filtre les departement dont aucun canton contient notre duel
    X= X[mask_dep]
    y = y[mask_dep]
    return (X[mask_canton], y[mask_canton])
                        


In [ ]:
#